In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

readdata

In [2]:
import numpy as np
import pandas as pd
import torch
import torchvision
from PIL import Image
import os
from torch.utils.data import Dataset


class myData(Dataset):
    def __init__(self,  rootDir, imgDir, labelDir):
        self.rootDir = rootDir # 根目录
        self.imgDir = os.path.join(self.rootDir, imgDir)
        self.imgPath = os.listdir(self.imgDir) # 所有图像路径列表
        self.labelDir = os.path.join(self.rootDir, labelDir)

    def __getitem__(self, idx): # idx为数据的序号（第几个数据）
        imgName = self.imgPath[idx]
        imgIndex = imgName[:-4]
        imgAbsName = os.path.join(self.imgDir, imgName)
        labelAbsName = os.path.join(self.labelDir, imgName)
        transform = torchvision.transforms.ToTensor()
        img = Image.open(imgAbsName).convert('L')
        img = transform(img)
        img = img
        label = Image.open(labelAbsName).convert('1')
        label = transform(label)
        return img, label

    def __len__(self):
        return len(self.imgPath)

class myData4Test(Dataset):
    def __init__(self,  rootDir, imgDir):
        self.rootDir = rootDir # 根目录
        self.imgDir = os.path.join(self.rootDir, imgDir)
        self.imgPath = os.listdir(self.imgDir) # 所有图像路径列表

    def __getitem__(self, idx): # idx为数据的序号（第几个数据）
        imgName = self.imgPath[idx]
        imgIndex = imgName[:-4]
        imgAbsName = os.path.join(self.imgDir, imgName)
        transform = torchvision.transforms.ToTensor()
        img = Image.open(imgAbsName).convert('L')
        img = img.resize((512, 512))
        img = transform(img)
        return img, imgName

    def __len__(self):
        return len(self.imgPath)


model

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

class VGGBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, middle_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(middle_channels)
        self.conv2 = nn.Conv2d(middle_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        return out

class Up(nn.Module):
    """Upscaling and concat"""

    def __init__(self):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
        diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return x


class NestedUNet(nn.Module):
    def __init__(self, num_classes=1, input_channels=1, deep_supervision=False, **kwargs):
        super().__init__()

        nb_filter = [32, 64, 128, 256, 512]

        self.deep_supervision = deep_supervision

        self.pool = nn.MaxPool2d(2, 2)
        self.up = Up()

        self.conv0_0 = VGGBlock(input_channels, nb_filter[0], nb_filter[0])
        self.conv1_0 = VGGBlock(nb_filter[0], nb_filter[1], nb_filter[1])
        self.conv2_0 = VGGBlock(nb_filter[1], nb_filter[2], nb_filter[2])
        self.conv3_0 = VGGBlock(nb_filter[2], nb_filter[3], nb_filter[3])
        self.conv4_0 = VGGBlock(nb_filter[3], nb_filter[4], nb_filter[4])

        self.conv0_1 = VGGBlock(nb_filter[0]+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_1 = VGGBlock(nb_filter[1]+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_1 = VGGBlock(nb_filter[2]+nb_filter[3], nb_filter[2], nb_filter[2])
        self.conv3_1 = VGGBlock(nb_filter[3]+nb_filter[4], nb_filter[3], nb_filter[3])

        self.conv0_2 = VGGBlock(nb_filter[0]*2+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_2 = VGGBlock(nb_filter[1]*2+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_2 = VGGBlock(nb_filter[2]*2+nb_filter[3], nb_filter[2], nb_filter[2])

        self.conv0_3 = VGGBlock(nb_filter[0]*3+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_3 = VGGBlock(nb_filter[1]*3+nb_filter[2], nb_filter[1], nb_filter[1])

        self.conv0_4 = VGGBlock(nb_filter[0]*4+nb_filter[1], nb_filter[0], nb_filter[0])

        if self.deep_supervision:
            self.final1 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final2 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final3 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final4 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
        else:
            self.final = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)


    def forward(self, input):
        x0_0 = self.conv0_0(input)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(self.up(x1_0, x0_0))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(self.up(x2_0, x1_0))
        x0_2 = self.conv0_2(self.up(x1_1, torch.cat([x0_0, x0_1], 1)))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(self.up(x3_0, x2_0))
        x1_2 = self.conv1_2(self.up(x2_1, torch.cat([x1_0, x1_1], 1)))
        x0_3 = self.conv0_3(self.up(x1_2, torch.cat([x0_0, x0_1, x0_2], 1)))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(self.up(x4_0, x3_0))
        x2_2 = self.conv2_2(self.up(x3_1, torch.cat([x2_0, x2_1], 1)))
        x1_3 = self.conv1_3(self.up(x2_2, torch.cat([x1_0, x1_1, x1_2], 1)))
        x0_4 = self.conv0_4(self.up(x1_3, torch.cat([x0_0, x0_1, x0_2, x0_3], 1)))

        if self.deep_supervision:
            output1 = self.final1(x0_1)
            output2 = self.final2(x0_2)
            output3 = self.final3(x0_3)
            output4 = self.final4(x0_4)
            return [output1, output2, output3, output4]

        else:
            output = self.final(x0_4)
            return output

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


train

output


In [4]:
import os
import numpy as np
import torch.utils.data
import torch
from PIL import Image
#from models import UNetPP
#from readData import myData4Test

rootDir = "/kaggle/input/gabseg/segmentation-plus"
testDir = "test/image"
testSet = myData4Test(rootDir, testDir)

testLoader = torch.utils.data.DataLoader(dataset= testSet, batch_size= 1, shuffle= False)

model =NestedUNet()

model.load_state_dict(torch.load("/kaggle/input/gmodel1/bestWeight2.pth"))
device = torch.device("cuda")
model = model.to(device)
testSize = len(testSet)
testLoss = 0
outDir = "/kaggle/working/label"
if not os.path.exists(outDir):
    os.mkdir(outDir)
i = 0
j = 0
model.eval()
with torch.no_grad():
    for data in testLoader:
        imgs, names = data
        imgs = imgs.to(device)
        outputs = model(imgs)
        for i in range(1):
            imgTensor = outputs[i]
            imgTensor[imgTensor > 0] = 255
            imgTensor[imgTensor <= 0] = 0
            imgAbsName = os.path.join(outDir, names[i])
            imgNP = np.array(imgTensor.data.cpu()[0],dtype=np.uint8)
            patch = Image.fromarray(imgNP)
            patch.save(imgAbsName)
        j += 1
        print("{} / {}".format(j, len(testLoader)))
# tensorboard --logdir=logs_train


1 / 40
2 / 40
3 / 40
4 / 40
5 / 40
6 / 40
7 / 40
8 / 40
9 / 40
10 / 40
11 / 40
12 / 40
13 / 40
14 / 40
15 / 40
16 / 40
17 / 40
18 / 40
19 / 40
20 / 40
21 / 40
22 / 40
23 / 40
24 / 40
25 / 40
26 / 40
27 / 40
28 / 40
29 / 40
30 / 40
31 / 40
32 / 40
33 / 40
34 / 40
35 / 40
36 / 40
37 / 40
38 / 40
39 / 40
40 / 40
